# Introduction

This is my vanilla RNN implementation.  I got helped from Dr.Karpathy of the Stanford 231n profs.  The github implementation can be found [here](https://gist.github.com/karpathy/d4dee566867f8291f086).  And the original blog post based on this can be found [here](http://karpathy.github.io/2015/05/21/rnn-effectiveness/).

This is an extension of RNN but with better gradient management.

I used the following resources:
- [Deriving LSTM Gradient for Backpropagation](https://wiseodd.github.io/techblog/2016/08/12/lstm-backprop/)
- [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
- [Karpathy Git Repo LSTM](https://gist.github.com/karpathy/587454dc0146a6ae21fc)
- [Vector,Matrix and Tensor Derivatives Cheatsheet](http://cs231n.stanford.edu/vecDerivs.pdf)
- [hackernoon: Understanding architecture of LSTM cell from scratch with code.](https://hackernoon.com/understanding-architecture-of-lstm-cell-from-scratch-with-code-8da40f0b71f4)
- [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
- [Simple LSTM](http://nicodjimenez.github.io/2014/08/08/lstm.html)
- [Simple LSTM Code](https://github.com/nicodjimenez/lstm/blob/master/lstm.py)
- [Only Numpy](http://blog.varunajayasiri.com/numpy_lstm.html)

# Imports

In [1]:
import numpy as np

# I/O

- get the data from a text file
- get the list of characters
- get the length of the db and the vocabulary
- associate each character with an index and vice versa

In [2]:
# Get the database from a list of characters that is found in the input.txt
# should be simple plain text file
data = open('input.txt', 'r').read() 

In [3]:
# get the list of characters by first identifying the set() of characters and place them in a list()
# NOTE: set() function here creates an unorders collection with no duplicate elements
chars = list(set(data))

In [22]:
print('The characters that were detected were:{}'.format(chars))

The characters that were detected were:['\n', ' ', "'", '-', 'A', 'F', 'I', 'H', 'N', 'P', 'S', 'T', 'W', 'Y', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y']


In [23]:
# obtain the size of the data and the size of the vocabulary we have
data_size, vocab_size = len(data), len(chars)

In [26]:
print 'data has {} characters, {} unique.'.format(data_size, vocab_size)

data has 1681 characters, 37 unique.


This is a text file so we know that there maybe 26 characters from the alphabet * 2 because of the capitals plus punctuations.  This seems reasonable so let us continue.

In [27]:
# enumerate the characters and give indices to them
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

In [29]:
char_to_ix

{'\n': 0,
 ' ': 1,
 "'": 2,
 '-': 3,
 'A': 4,
 'F': 5,
 'H': 7,
 'I': 6,
 'N': 8,
 'P': 9,
 'S': 10,
 'T': 11,
 'W': 12,
 'Y': 13,
 'a': 14,
 'b': 16,
 'c': 15,
 'd': 18,
 'e': 17,
 'f': 20,
 'g': 19,
 'h': 22,
 'i': 21,
 'j': 24,
 'k': 23,
 'l': 26,
 'm': 25,
 'n': 28,
 'o': 27,
 'p': 29,
 'r': 31,
 's': 30,
 't': 33,
 'u': 32,
 'v': 35,
 'w': 34,
 'y': 36}

In [30]:
ix_to_char

{0: '\n',
 1: ' ',
 2: "'",
 3: '-',
 4: 'A',
 5: 'F',
 6: 'I',
 7: 'H',
 8: 'N',
 9: 'P',
 10: 'S',
 11: 'T',
 12: 'W',
 13: 'Y',
 14: 'a',
 15: 'c',
 16: 'b',
 17: 'e',
 18: 'd',
 19: 'g',
 20: 'f',
 21: 'i',
 22: 'h',
 23: 'k',
 24: 'j',
 25: 'm',
 26: 'l',
 27: 'o',
 28: 'n',
 29: 'p',
 30: 's',
 31: 'r',
 32: 'u',
 33: 't',
 34: 'w',
 35: 'v',
 36: 'y'}

# Parameters

## HyperParameters

In [31]:
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

## Model Parameters

In [39]:
# model parameters
concatHiddenLayerInputLayerSize = vocab_size + hidden_size

Wih=np.random.randn(concatHiddenLayerInputLayerSize, hidden_size) 
Wfh=np.random.randn(concatHiddenLayerInputLayerSize, hidden_size) 
Woh=np.random.randn(concatHiddenLayerInputLayerSize, hidden_size) 
Wgh=np.random.randn(concatHiddenLayerInputLayerSize, hidden_size) 
Wch=np.random.randn(concatHiddenLayerInputLayerSize, hidden_size) 
Wyh=np.random.randn(hidden_size, vocab_size) 

bi=np.zeros((1, hidden_size))
bf=np.zeros((1, hidden_size))
bo=np.zeros((1, hidden_size))
bg=np.zeros((1, hidden_size))
bc=np.zeros((1, hidden_size))
by=np.zeros((1, hidden_size))


# Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
# Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
# Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
# bh = np.zeros((hidden_size, 1)) # hidden bias
# by = np.zeros((vocab_size, 1)) # output bias

# Functions

## Activation Functions

### Sigmoid

In [34]:
def sigmoid(x):
    sigmoidOut = 1 / (1 + np.exp(-x))    
    return sigmoidOut  

In [35]:
def dsigmoid(x):
    dsigmoidOut = sigmoid(x)*(1 - sigmoid(x))
    return dsigmoidOut  

### Tanh

In [36]:
def tanh(x):
    tanhOut = np.tanh(x)
    return tanhOut

In [38]:
def dtanh(x):
    dtanh = 1 - tanh(x)**2

## Loss

**Forward Pass**
![](https://i.imgur.com/qaJLCZK.png)

In [ ]:
def lossFun(inputs, targets, hprev):
    """
    inputs,targets are both list of integers of size seq_length 
    hprev is [Hx1] array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    # inits for forward pass
    # xs: input state
    # hs: hidden state
    # ys: output state
    # ps: propbability state
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0

    # forward pass
    # t is the tick for all inputs in the sequence
    for t in xrange(len(inputs)):
        # one-hot-encoder
        # get zeros
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        # place a one where the input index is
        xs[t][inputs[t]] = 1

        # obtain the current hidden state by using the regular equation for RNN
        # ht = tanh(whh*ht-1 + wxh*xt) where * is a dot operation
        # in other words 
        # multiply the weigths with the previous hidden state, 
        # add the weigths times the input of the current tick, 
        # add some bias
        # apply a non-linarity in this case tanh
        # this produces the current hidden state
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state

        # obtain the output stae by applying the dot product of the input weigths with the hidden state and adding a bias
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars

        # loss function which is the negative log cross-entropy
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)

    # inits for backward pass
    # backprop: compute gradients going backwards
    # dWxh: the gradient of the weights between the input and hidden layer
    # dWhh: the gradient of the weights in the hidden layer
    # dWhy: the gradient of the weights between the hidden and output layer
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    # dbh: the bias gradient of the hidden layer
    # dby: the bias gradient of the output layer
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    # dhnext: the running gradient while propagarting through time.  If you don't see it refer to the computational graph.
    dhnext = np.zeros_like(hs[0])

    # going backwards for each t is the tick for all inputs in the sequence. Again look at the computational graph.
    for t in reversed(xrange(len(inputs))):
        # upstream through the loss function
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here

        # upstream through the weigths    
        dWhy += np.dot(dy, hs[t].T)
        dby += dy    

        # upstream through tanh (derivative of tanh)
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw

        # upstream through stack (see computational graph)
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)

        # upstream thought the next input
        dhnext = np.dot(Whh.T, dhraw)

    # Mitigate exploding and vanishing gradients.
    # if the largest singular value > 1 in any of the gradients then you would slowly increase to infinity and explode
    # if the largest singular value < 1 in any of the gradients then you would slowly decrease to 0 and disappear
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients

    # return the following
    # [loss]: for logging
    # [dWxh, dWhh, dWhy, dbh, dby]: grardients for input,hidden,output weigts and biases
    # [hs]: the current hidden state which will be the previous state for the next set of sequence in the backprop thourgh time
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

## Sample
Get sample from the output distribution

In [ ]:
def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is the current memory state, 
  seed_ix is seed letter for first time step ... basically the starting seed to generate the text
  n is the number of characters to generate
  """
  
  # intialization  
  # one-hot encode the seed
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1

  # running array list for the outputs
  ixes = []

  # go through each tick to generate the next character in all of the n generated characters  
  for t in xrange(n):
    # forward pass
    
    # calculate hidden state
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    
    # calculate output
    y = np.dot(Why, h) + by
    
    # calculate probabilities
    p = np.exp(y) / np.sum(np.exp(y))
    
    # select a random with some probability associated with values
    # np.random.choice(allPossibileOutcomes, probabilityOfTheOutcomes)
    # choose a number in the vocab based on the probabilty distribution found in previous step
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    
    # one-hot encode this in a vector
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    
    # append running vector outputs
    ixes.append(ix)
  return ixes

In [ ]:
# def sample(h, seed_ix, n):
#   """ 
#   sample a sequence of integers from the model 
#   h is memory state, seed_ix is seed letter for first time step
#   """
#   x = np.zeros((vocab_size, 1))
#   x[seed_ix] = 1
#   ixes = []
#   for t in xrange(n):
#     h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
#     y = np.dot(Why, h) + by
#     p = np.exp(y) / np.sum(np.exp(y))
#     ix = np.random.choice(range(vocab_size), p=p.ravel())
#     x = np.zeros((vocab_size, 1))
#     x[ix] = 1
#     ixes.append(ix)
#   return ixes

# RNN

## init

### Pointers

In [ ]:
# keep track of the data index as well as the iteration 
# n = interation counter
# p = data pointer
n, p = 0, 0

### Weights

In [40]:
# initialize the weights all to zero
Wih=np.random.randn(concatHiddenLayerInputLayerSize, hidden_size) 
Wfh=np.random.randn(concatHiddenLayerInputLayerSize, hidden_size) 
Woh=np.random.randn(concatHiddenLayerInputLayerSize, hidden_size) 
Wgh=np.random.randn(concatHiddenLayerInputLayerSize, hidden_size) 
Wch=np.random.randn(concatHiddenLayerInputLayerSize, hidden_size) 
Wyh=np.random.randn(hidden_size, vocab_size) 

mWih = np.zeros_like(Wih)
mWfh = np.zeros_like(Wfh)
mWoh = np.zeros_like(Woh)
mWgh = np.zeros_like(Wgh)
mWch = np.zeros_like(Wch)
mWyh = np.zeros_like(Wyh)

### Optimizer

Here we are using Adagrad so we need to initialize the cache and the eps.

![AdaGradImplementation](https://i.imgur.com/p9IvCME.png)

In [41]:
# Bias
mbi = np.zeros_like(bi)
mbf = np.zeros_like(bf)
mbo = np.zeros_like(bo)
mbg = np.zeros_like(bg)
mbc = np.zeros_like(bc)
mby = np.zeros_like(by)

### Loss

In [42]:
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

## RNN Start

Note: this will run forever until you terminate.  Maybe later if I have some time I could add a logging system and maybe a termination loops. For now this is good just to see how well things work.

![](https://i.imgur.com/7OpZrWz.png)
![image](https://i.imgur.com/EwjRQ0C.png) 
![](https://i.imgur.com/jzIcL6x.png)

In [ ]:
while True:   
  
  # Step #1 Truncated Backprop through time paradigm
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  # Because we are doing truncated backprop thorugh time (by 25 steps) we need to see if we get an overflow.  If so then we need
  # to initialize everything back to zero.  this means that the previous hidden state is now 0 and the data pointer is also back
  # to zero
  if p+seq_length+1 >= len(data) or n == 0: 
    # reset RNN memory
    hprev = np.zeros((hidden_size,1)) 
    # go from start of data
    p = 0 #

  # Step #2 - Obtain Inputs
  # Get the sequence of inputs in the database with length seq_length
  # input is a list of indicies in the char_to_ix
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]

  # Step #3 - Obtain Outputs
  # Get the sequence of outputs in the database with length seq_length
  # basically the next character of the input
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # Step #4 - Backprop and calculate loss
  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  
  # Step #5 - Parameter updates 
  # perform parameter update with Adagrad
  # This is an efficient way of looping through all the parameters that needs to be update
  # zip each group of parameter types using the zip function.  
  # So the zip function places zip([parametsr],[derivatives of the parameters],[memory of the paramters])
  # loop through them so in this case it will first take the Wxh i.e. the first terms in each of the zipped lists.
  # In other words 
  # param = Wxh 
  # dparam = dWxh 
  # mem = mWxh
  # then you apply the adagra update rule
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                            [dWxh, dWhh, dWhy, dbh, dby], 
                            [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  # Step #6 - Randoomly sample what the model is generating
  # sample from the model now and then by outputing what it generated and also show the loss
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print '----\n %s \n----' % (txt, )
    print 'iter %d, loss: %f' % (n, smooth_loss) 
    

  # Step #7 - increment the iteration and also the step sequence through the database.  
  p += seq_length # move data pointer
  n += 1 # iteration counter 